# Prediciting Post Reaction using Random Forest

## Import Libraries

### Main Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Scikit Learn

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score

## Load Dataset

In [3]:
df_train = pd.read_csv('postSenti_train.tsv', index_col='Unnamed: 0', sep='\t')
df_test = pd.read_csv('postSenti_test.tsv', index_col='Unnamed: 0', sep='\t') 

In [4]:
# Separate features and target for training dataset
X_train = df_train.drop(['sentiment_class'], axis=1)
y_train = df_train['sentiment_class']

In [5]:
# Separate features and target for testing dataset
X_test = df_test.drop(['sentiment_class'], axis=1)
y_test = df_test['sentiment_class']

## Random Forest Classification

### Feature selection using RandomForestClassifier--feature_importances

In [6]:
# Instantiate RandomForestClassifier
rf = RandomForestClassifier(random_state=35).fit(X_train, y_train)
rf_feat_imp = rf.feature_importances_
# Feature importance
feature_importance = pd.DataFrame(rf_feat_imp, index=X_train.columns, columns=["Feat_Imp"])

In [7]:
#Sort in descending order to get Most Important to Least Important
feature_importance.sort_values("Feat_Imp",ascending=False, inplace=True)
feature_importance.reset_index(inplace=True)
feature_importance

,index,Feat_Imp
0,Topic 0,0.167904
1,hour_day,0.139367
2,Topic 1,0.109699
3,Topic 2,0.069233
4,Topic 7,0.067534
5,Topic 9,0.063990
6,Topic 5,0.060787
7,Topic 6,0.057733
8,Topic 8,0.055409
9,Topic 4,0.051431


In [8]:
# Instantiate Random Forest Classifier
clf = RandomForestClassifier()

# Function to get the accuracy of the model
def get_accuracy(xtrain, ytrain, xtest, ytest):
    clf.fit(xtrain, ytrain)
    ypred = clf.predict(xtest)
    return accuracy_score(ypred, ytest)*100

In [9]:
score_dict = {}
# Store accuracies in score_dict
for i in range(1, len(feature_importance)):
    features = feature_importance.iloc[:i]
    predictor = list(features['index'])
    key = 'top' + str(i)
    score_dict[key] = get_accuracy(X_train[predictor], y_train, X_test[predictor], y_test)

In [10]:
# Check scores to determine how many features will be used
score_dict

{'top1': 46.42857142857143,
 'top2': 50.51020408163265,
 'top3': 54.59183673469388,
 'top4': 50.51020408163265,
 'top5': 59.183673469387756,
 'top6': 55.10204081632652,
 'top7': 55.61224489795919,
 'top8': 56.12244897959183,
 'top9': 56.63265306122449,
 'top10': 56.12244897959183,
 'top11': 56.12244897959183,
 'top12': 59.183673469387756,
 'top13': 57.14285714285714,
 'top14': 59.183673469387756,
 'top15': 59.183673469387756,
 'top16': 59.183673469387756,
 'top17': 61.73469387755102}

In [11]:
# Top 17 Feature Importance
top_feature = feature_importance.iloc[:17]
predictor = list(top_feature['index'])
predictor

['Topic 0',
 'hour_day',
 'Topic 1',
 'Topic 2',
 'Topic 7',
 'Topic 9',
 'Topic 5',
 'Topic 6',
 'Topic 8',
 'Topic 4',
 'Topic 3',
 'Promo',
 'Engagement',
 'photo',
 'App Update',
 'link',
 'video']

### Hyperparameter Optimization using GridSearch

In [12]:
param_grid = {'min_samples_split':[2,5,10],
              'criterion':['gini','entropy'],
              'min_samples_leaf':[1,2,4],
              'n_estimators':[200,400,600,800,1000],
              'max_depth':[20,40,60,80,100,None]}

In [13]:
# Instantiate RandomizedSearchCV
grid_search = RandomizedSearchCV(clf, param_distributions=param_grid, cv=5, random_state=0)

In [14]:
# Fit X_train
grid_search.fit(X_train[predictor], y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'min_samples_split': [2, 5, 10], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000], 'max_depth': [20, 40, 60, 80, 100, None]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [15]:
print("Best estimator:")
print(grid_search.best_estimator_)
print("Best parameters:")
print(grid_search.best_params_)
print("Best score:")
print(grid_search.best_score_)

Best estimator:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Best parameters:
{'n_estimators': 800, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 100, 'criterion': 'entropy'}
Best score:
0.5659411011523687


### Build Random Forest Model

In [16]:
# Instantiate SVC
rf_clf = RandomForestClassifier(n_estimators=800, criterion='entropy', min_samples_leaf=4, min_samples_split=10, max_depth=100)
# Fit the training dataset on the SVC
rf_clf.fit(X_train[predictor], y_train)
# Predict the labels on validation dataset
y_pred = rf_clf.predict(X_test[predictor])

### Evaluate Random Forest Model

In [17]:
# Get Metrics
accuracy = accuracy_score(y_pred, y_test)*100
# Store metrics in dictionary
score_dict = {'Random Forest': pd.Series([accuracy], index=['AccuracyScore'])}
# Transform dictionary to df
df_metrics = pd.DataFrame(score_dict)
df_metrics

,Random Forest
AccuracyScore,65.306122


## Cross Validation

In [18]:
print('Mean Cross validated score -> ', np.mean(cross_val_score(rf_clf, X_train[predictor], y_train, cv=5)*100))

Mean Cross validated score ->  56.85299075499157
